<a href="https://colab.research.google.com/github/shashankdeshpande/llm-examples/blob/master/OpenAI-API-Function-Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TOC:
* [OpenAI - Function calling capability](#chapter1)
* [Multiple / Sequential Function Calling - OpenAI API](#chapter2)

# <a name="chapter1"></a>OpenAI - Function calling capability

OpenAI has recently introduced a new capability called function calling, which allows developers to retrieve structured outputs from the GPT-4 and GPT-3.5 models.

This feature enables chatbots to leverage external tools, transform natural language into database queries, and extract structured data from text. The models have been fine-tuned to identify instances where a function should be invoked and provide JSON responses that align with the function arguments. Function calling allows developers to more reliably get structured data back from the model

**Reference** - https://openai.com/blog/function-calling-and-other-api-updates

In [ ]:
!pip install -q openai dateparser

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.5 MB/s eta 0:00:00


In [ ]:
import json
import openai
import logging
import requests
import traceback
import dateparser

In [ ]:
logger = logging.getLogger('OpenAI-Example')
logger.setLevel(logging.INFO)

In [ ]:
# Get your key here - https://platform.openai.com/account/api-keys

openai.api_key = "YOUR_OPENAI_KEY_HERE"

## Step 1 - Create a custom function
In this step, we will be creating a personalized function that will retrieve country-specific trends upon request.

We are referring google trends from here - https://trends.google.com/trends/trendingsearches/daily?geo=IN&hl=en-GB

In [ ]:
def get_google_trends(country_code, date_string='today'):
    """Method to retrieve country specific trends

    Args:
        country_code (str): alpha-2 country code, example - IN,US,UK
        date_string (str): date string supported by dateparser, example - today, yesterday

    Returns:
        list: list of dict having trend query and traffic details
    """
    trends = []
    try:
        logger.info(f'received args - country_code: {country_code}, date_string: {date_string}')
        dt = dateparser.parse(date_string).strftime('%Y%m%d')
        url = 'https://trends.google.com/trends/api/dailytrends'
        params = {'hl':'en-GB','tz':-330, 'geo': country_code, 'ns':15, 'ed': dt}
        response = requests.get(url, params=params, timeout=5)
        if response.status_code == 200:
            tinfo = json.loads(response.content[5:])
            tinfo = tinfo['default']['trendingSearchesDays'][0]
            if tinfo['date'] == dt:
                for i in tinfo['trendingSearches']:
                    trends.append({
                        'query': i['title']['query'],
                        'traffic': i['formattedTraffic']
                    })
    except Exception as e:
        logger.error('Error in get_trends: \n {}'.format(traceback.format_exc()))
    return trends

In [ ]:
get_google_trends('IN')

INFO:OpenAI-Example:received args - country_code: IN, date_string: today


[{'query': 'Messi', 'traffic': '100K+'},
 {'query': 'Wagner Group', 'traffic': '20K+'},
 {'query': 'Lionel Messi', 'traffic': '20K+'},
 {'query': 'Rahul Gandhi', 'traffic': '20K+'},
 {'query': 'Ruturaj Gaikwad', 'traffic': '10K+'},
 {'query': 'TSPSC Group 4 Hall Ticket', 'traffic': '10K+'},
 {'query': 'Mary Millben', 'traffic': '10K+'},
 {'query': 'Lionel Messi Birthday', 'traffic': '5K+'}]

## Step 2 - Specify necessary function details as required by OpenAI
Make sure to mention clear and detailed description for function as well as arguments required by that function. This will assist the OpenAI model in determining the appropriate usage of the function and aid in extracting the essential entities from user queries.

We can have details of multiple functions, currently experimenting with only one.

In [ ]:
custom_function_info = [
    {
        "name": "get_google_trends",
        "description": "Get list of trending queries along with traffic observed",
        "parameters": {
            "type": "object",
            "properties": {
                "country_code": {
                    "type": "string",
                    "description": "alpha-2 country code",
                },
                "date_string": {
                    "type": "string",
                    "description": "date string supported by dateparser"
                    },
            },
            "required": ["country_code", "date_string"],
        },
    }
]

## Step 3 - Call OpenAI API with custom function details

In [ ]:
user_query = "Show me current google trends in India"

openai_model = "gpt-3.5-turbo-0613"

response = openai.ChatCompletion.create(
    model=openai_model,
    messages=[{"role": "user", "content": user_query}],
    functions=custom_function_info,
    function_call="auto", #OpenAI will determine whether a custom function is necessary or not based on query, otherwise can set this as {"name": "get_google_trends"}
)

In [ ]:
response

<OpenAIObject chat.completion id=chatcmpl-7Utl9iv3YiXanW1Rd6dYl07TQBoKz at 0x7fb650c837e0> JSON: {
  "id": "chatcmpl-7Utl9iv3YiXanW1Rd6dYl07TQBoKz",
  "object": "chat.completion",
  "created": 1687598535,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_google_trends",
          "arguments": "{\n  \"country_code\": \"IN\",\n  \"date_string\": \"today\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 79,
    "completion_tokens": 26,
    "total_tokens": 105
  }
}

## Step 4 - Retrieve identified arguments from the response and call the custom function

In [ ]:
identified_func = response.choices[0].message.to_dict()
print('Response function call: ', identified_func)

identified_func_args = identified_func['function_call']['arguments']
identified_func_args = json.loads(identified_func_args)
print('Function arguments: ', identified_func_args)

Response function call:  {'role': 'assistant', 'content': None, 'function_call': <OpenAIObject at 0x7fb650d9fec0> JSON: {
  "name": "get_google_trends",
  "arguments": "{\n  \"country_code\": \"IN\",\n  \"date_string\": \"today\"\n}"
}}
Function arguments:  {'country_code': 'IN', 'date_string': 'today'}


In [ ]:
func_output = get_google_trends(**identified_func_args)
func_output

INFO:OpenAI-Example:received args - country_code: IN, date_string: today


[{'query': 'Messi', 'traffic': '100K+'},
 {'query': 'Wagner Group', 'traffic': '20K+'},
 {'query': 'Lionel Messi', 'traffic': '20K+'},
 {'query': 'Rahul Gandhi', 'traffic': '20K+'},
 {'query': 'Ruturaj Gaikwad', 'traffic': '10K+'},
 {'query': 'TSPSC Group 4 Hall Ticket', 'traffic': '10K+'},
 {'query': 'Mary Millben', 'traffic': '10K+'},
 {'query': 'Lionel Messi Birthday', 'traffic': '5K+'}]

## Step 5: Call OpenAI with function output and query to get answer

In [ ]:
final_response = openai.ChatCompletion.create(
    model=openai_model,
    messages=[
        {"role": "user", "content": user_query},
        {"role": "assistant", "content": None, "function_call": {"name": "get_current_weather", "arguments": json.dumps(identified_func_args)}},
        {"role": "function", "name": "get_google_trends", "content": json.dumps(func_output)}
        ],
    functions=custom_function_info
)

In [ ]:
final_response

<OpenAIObject chat.completion id=chatcmpl-7UtlQ3BqPFyZ7XNeYczOPfpSGBR6M at 0x7fb651dcb470> JSON: {
  "id": "chatcmpl-7UtlQ3BqPFyZ7XNeYczOPfpSGBR6M",
  "object": "chat.completion",
  "created": 1687598552,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Here are the current Google trends in India:\n\n1. Messi - 100K+\n2. Wagner Group - 20K+\n3. Lionel Messi - 20K+\n4. Rahul Gandhi - 20K+\n5. Ruturaj Gaikwad - 10K+\n6. TSPSC Group 4 Hall Ticket - 10K+\n7. Mary Millben - 10K+\n8. Lionel Messi Birthday - 5K+"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 252,
    "completion_tokens": 94,
    "total_tokens": 346
  }
}

In [ ]:
print(final_response.choices[0]['message']['content'])

Here are the current Google trends in India:

1. Messi - 100K+
2. Wagner Group - 20K+
3. Lionel Messi - 20K+
4. Rahul Gandhi - 20K+
5. Ruturaj Gaikwad - 10K+
6. TSPSC Group 4 Hall Ticket - 10K+
7. Mary Millben - 10K+
8. Lionel Messi Birthday - 5K+


## Consolidating all these elements ..
Creating a separate function which will return respond to user query

In [ ]:
def ask_openai(user_query):
    answer = ''
    response = openai.ChatCompletion.create(
        model=openai_model,
        messages=[{"role": "user", "content": user_query}],
        functions=custom_function_info,
        function_call="auto"
    )
    resp = response.choices[0].message.to_dict()
    if 'function_call' in resp:
        identified_func_args = json.loads(resp['function_call']['arguments'])
        func_output = get_google_trends(**identified_func_args)
        final_response = openai.ChatCompletion.create(
            model=openai_model,
            messages=[
                {"role": "user", "content": user_query},
                {"role": "assistant", "content": None, "function_call": {"name": resp['function_call']['name'], "arguments": json.dumps(identified_func_args)}},
                {"role": "function", "name": resp['function_call']['name'], "content": json.dumps(func_output)}
                ],
            functions=custom_function_info
        )
        answer = final_response.choices[0]['message']['content']
    else:
        answer = resp['content']
    return answer

In [ ]:
# In this example, OpenAI correctly identifies country which user was referring to, and also provide additional information by utilizing function output

answer = ask_openai('Which is the most popular search trend in the nation with the largest democracy. Tell me more about that')
print(answer)

INFO:OpenAI-Example:received args - country_code: IN, date_string: today


The most popular search trend in India, which is the nation with the largest democracy, is "Messi". This search term has a traffic volume of over 100K+ searches today. 

"Messi" refers to Lionel Messi, the Argentine professional football player, who is considered one of the greatest footballers of all time. He has a massive fan following around the world, including in India. People may be searching for updates on his latest matches, goals, news, or other information related to his career.

It's worth noting that other popular search trends in India today include "Wagner Group" and "Rahul Gandhi," each with a traffic volume of 20K+ searches. Wagner Group is a private military company, and Rahul Gandhi is an Indian politician and the former president of the Indian National Congress party.

Overall, "Messi" is currently the most popular search trend in India, reflecting the immense popularity and interest in football in the country.


In [ ]:
answer = ask_openai('Could you provide information on the topic that was trending in the US yesterday and share the traffic data for the most popular trend?')
print(answer)

INFO:OpenAI-Example:received args - country_code: US, date_string: yesterday


The following topics were trending in the US yesterday:

1. Russia - 500K+ traffic
2. Kim Petras - 100K+ traffic
3. Frozen fruit recall - 50K+ traffic
4. Reds - 50K+ traffic
5. Cincinnati Reds - 50K+ traffic
6. Evil Dead Rise - 50K+ traffic
7. Jordan Walsh - 50K+ traffic
8. Damian Lillard - 20K+ traffic
9. Lonzo Ball - 20K+ traffic
10. Taylor Swift Minneapolis - 20K+ traffic
11. Highlands Ranch tornado - 20K+ traffic
12. Marcus Sasser - 20K+ traffic
13. Lab-grown meat - 20K+ traffic
14. Knicks - 20K+ traffic
15. Adam Rich - 20K+ traffic
16. Nick Smith Jr - 20K+ traffic

The most popular trend based on traffic was "Russia" with over 500K searches.


In [ ]:
# In this example, custom function is not utilized which is correct

answer = ask_openai('Can you explain blockchain in simple terms')
print(answer)

Certainly! Blockchain is a technology that allows multiple parties to maintain a secure and transparent record of transactions or information without the need for a central authority, such as a bank or government.

Imagine a digital ledger or spreadsheet that is shared and continuously updated across a network of computers called nodes. Each node has a copy of this ledger and verifies the transactions or information recorded on it.

When a new transaction or piece of information needs to be added to the ledger, it is grouped together with other transactions into a block. Before this block is added to the chain, it goes through a process called "consensus." This means that the nodes in the network agree that the information in the block is valid and should be added.

Once a block is approved and added to the chain, it becomes a permanent part of the ledger and cannot be altered or deleted. Each block also contains a reference to the previous block, creating a chain of linked blocks.

Th

# <a name="chapter2"></a>Multiple / Sequential Function Calling - OpenAI API

Exploring capabilities of `function_call` usage for recurrent/sequential functions -

For the purpose of understanding, we will utilize the contents of the `sample_folder` and experiment with the folder's contents.

In [ ]:
!pip install -q humanfriendly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.0 MB/s eta 0:00:00


In [ ]:
import os
import time
import shutil
import traceback
import humanfriendly
from tqdm import tqdm

def get_folder_contents(path):
    contents = []
    try:
        for file in os.listdir(path):
            size_bytes = os.stat(os.path.join(path, file)).st_size
            size = humanfriendly.format_size(size_bytes)
            contents.append({
                'file_name': file,
                'size': size
            })
    except Exception as e:
        logger.error('Error in get_folder_contents:\n {}'.format(traceback.format_exc()))
    return contents

def create_folder(name):
    if not os.path.exists(name):
        os.makedirs(name)
        msg = f'Folder creation succesfull: {name}'
    else:
        msg = f'Folder already exists: {name}'
    return msg

def copy_file(src, dest):
    if not os.path.exists(dest):
        os.makedirs(dest)
    shutil.copy(src, dest)
    return f'Copy successfull: {src} to {dest}'

In [ ]:
get_folder_contents('/content/sample_data/')

[{'file_name': 'anscombe.json', 'size': '1.7 KB'},
 {'file_name': 'README.md', 'size': '930 bytes'},
 {'file_name': 'mnist_test.csv', 'size': '18.29 MB'},
 {'file_name': 'mnist_train_small.csv', 'size': '36.52 MB'},
 {'file_name': 'california_housing_train.csv', 'size': '1.71 MB'},
 {'file_name': 'california_housing_test.csv', 'size': '301.14 KB'}]

In [ ]:
func_name_map = {
    'get_folder_contents': get_folder_contents,
    'create_folder': create_folder,
    'copy_file': copy_file
    }

In [ ]:
custom_function_info = [
    {
        "name": "get_folder_contents",
        "description": "Use this function to obtain a list of files present in a specified folder, including their respective sizes in human readable format",
        "parameters": {
            "type": "object",
            "properties": {
                "path": {
                    "type": "string",
                    "description": f"Full path of the folder, should start with {os.getcwd()}",
                }
            },
            "required": ["path"],
        },
    },
    {
        "name": "create_folder",
        "description": "Use this function to create new folder",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "name of new folder, don't forget to replace all spaces with underscores",
                }
            },
            "required": ["name"],
        },
    },
    {
        "name": "copy_file",
        "description": "Use this function to copy file to specified destination",
        "parameters": {
            "type": "object",
            "properties": {
                "src": {
                    "type": "string",
                    "description": "full source path of the file which is to be copied",
                },
                "dest": {
                    "type": "string",
                    "description": "full path of the destination folder. Make sure not include file name as folder",
                }
            },
            "required": ["src","dest"],
        },
    }
]

In [ ]:
def ask_openai(user_query, openai_model="gpt-3.5-turbo-0613"):
    answer = ''
    messages = [
        {"role": "system","content": """
                You are intelligent chatbot.
                You are only allowed to use specified functions with relevant order by looking at the user query and context.
                You don’t have a python function available, never attempt to call python
                """},
        {"role": "user", "content": user_query}
        ]
    while True:
        logger.info(f'Calling API: {messages}')
        response = openai.ChatCompletion.create(
            model=openai_model,
            messages=messages,
            functions=custom_function_info,
            function_call='auto'
        )
        resp = response.choices[0].message.to_dict()
        logger.info(f'API Response: {resp}')
        if 'function_call' in resp:
            func_call_resp = resp['function_call']
            identified_func = func_call_resp['name']
            identified_func_args = json.loads(func_call_resp['arguments'])
            func_output = func_name_map[identified_func](**identified_func_args)
            messages.extend([
                {"role": "assistant", "content": None, "function_call": {"name": identified_func, "arguments": json.dumps(identified_func_args)}},
                {"role": "function", "name": identified_func, "content": json.dumps(func_output)},

                {"role": "user", 'content': f"User query was - {user_query}. The previous function returned {json.dumps(func_output)}. Suggest next steps if any as per user query and context, else respond with relevant answer"}
            ])
        else:
            break

        # added this to avoid RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues.
        sleep_seconds = 30
        for i in tqdm(range(sleep_seconds), desc='Sleeping'):
            time.sleep(1)

    answer = resp['content']
    return answer

In [ ]:
# original folder structure

!ls -lR

.:
total 4
drwxr-xr-x 1 root root 4096 Jun 23 01:15 sample_data

./sample_data:
total 55504
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root   301141 Jun 23 01:15 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Jun 23 01:15 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Jun 23 01:15 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Jun 23 01:15 mnist_train_small.csv
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md


### Example 1: Single function usage

In [ ]:
user_query = "Which file is the largest in the sample_data folder? "
answer = ask_openai(user_query)
print(answer)

INFO:OpenAI-Example:Calling API: [{'role': 'system', 'content': '\n                You are intelligent chatbot.\n                You are only allowed to use specified functions with relevant order by looking at the user query and context.\n                You don’t have a python function available, never attempt to call python\n                '}, {'role': 'user', 'content': 'Which file is the largest in the sample_data folder? '}]
INFO:OpenAI-Example:API Response: {'role': 'assistant', 'content': None, 'function_call': <OpenAIObject at 0x7f1cb41214e0> JSON: {
  "name": "get_folder_contents",
  "arguments": "{\n\"path\": \"/content/sample_data\"\n}"
}}
Sleeping: 100%|██████████| 30/30 [00:30<00:00,  1.00s/it]
INFO:OpenAI-Example:Calling API: [{'role': 'system', 'content': '\n                You are intelligent chatbot.\n                You are only allowed to use specified functions with relevant order by looking at the user query and context.\n                You don’t have a python f

The largest file in the sample_data folder is "mnist_train_small.csv" with a size of 36.52 MB.


### Example 2: Single function recurrent usage

In [ ]:
user_query = "Create two new folder namely new_folder1 and new_folder2"
answer = ask_openai(user_query)
print(answer)

INFO:OpenAI-Example:Calling API: [{'role': 'system', 'content': '\n                You are intelligent chatbot.\n                You are only allowed to use specified functions with relevant order by looking at the user query and context.\n                You don’t have a python function available, never attempt to call python\n                '}, {'role': 'user', 'content': 'Create two new folder namely new_folder1 and new_folder2'}]
INFO:OpenAI-Example:API Response: {'role': 'assistant', 'content': None, 'function_call': <OpenAIObject at 0x7f1cb4176cf0> JSON: {
  "name": "create_folder",
  "arguments": "{\"name\": \"new_folder1\"}"
}}
Sleeping: 100%|██████████| 30/30 [00:30<00:00,  1.00s/it]
INFO:OpenAI-Example:Calling API: [{'role': 'system', 'content': '\n                You are intelligent chatbot.\n                You are only allowed to use specified functions with relevant order by looking at the user query and context.\n                You don’t have a python function availabl

You have successfully created the two new folders - new_folder1 and new_folder2. Is there anything else I can assist you with?


In [ ]:
# modified folder structure

!ls -lR

.:
total 12
drwxr-xr-x 2 root root 4096 Jun 26 14:10 new_folder1
drwxr-xr-x 2 root root 4096 Jun 26 14:11 new_folder2
drwxr-xr-x 1 root root 4096 Jun 23 01:15 sample_data

./new_folder1:
total 0

./new_folder2:
total 0

./sample_data:
total 55504
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root   301141 Jun 23 01:15 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Jun 23 01:15 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Jun 23 01:15 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Jun 23 01:15 mnist_train_small.csv
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md


### Example 3: Complex query - invloves multiple and recurrent function usage

In [ ]:
user_query = "Create two new folder namely big-file and small-file. Then copy largest file from sample_data folder to big-file and smallest to small-file"
answer = ask_openai(user_query)
print(answer)

INFO:OpenAI-Example:Calling API: [{'role': 'system', 'content': '\n                You are intelligent chatbot.\n                You are only allowed to use specified functions with relevant order by looking at the user query and context.\n                You don’t have a python function available, never attempt to call python\n                '}, {'role': 'user', 'content': 'Create two new folder namely big-file and small-file. Then copy largest file from sample_data folder to big-file and smallest to small-file'}]
INFO:OpenAI-Example:API Response: {'role': 'assistant', 'content': None, 'function_call': <OpenAIObject at 0x7f1cb42a6520> JSON: {
  "name": "create_folder",
  "arguments": "{\n  \"name\": \"big-file\"\n}"
}}
Sleeping: 100%|██████████| 30/30 [00:30<00:00,  1.00s/it]
INFO:OpenAI-Example:Calling API: [{'role': 'system', 'content': '\n                You are intelligent chatbot.\n                You are only allowed to use specified functions with relevant order by looking at 

The task has been completed successfully. The largest file from the sample_data folder has been copied to the big-file folder, and the smallest file has been copied to the small-file folder. Is there anything else I can help you with?


In [ ]:
# modified folder structure

!ls -lR

.:
total 20
drwxr-xr-x 2 root root 4096 Jun 26 14:16 big-file
drwxr-xr-x 2 root root 4096 Jun 26 14:10 new_folder1
drwxr-xr-x 2 root root 4096 Jun 26 14:11 new_folder2
drwxr-xr-x 1 root root 4096 Jun 23 01:15 sample_data
drwxr-xr-x 2 root root 4096 Jun 26 14:17 small-file

./big-file:
total 35668
-rw-r--r-- 1 root root 36523880 Jun 26 14:16 mnist_train_small.csv

./new_folder1:
total 0

./new_folder2:
total 0

./sample_data:
total 55504
-rwxr-xr-x 1 root root     1697 Jan  1  2000 anscombe.json
-rw-r--r-- 1 root root   301141 Jun 23 01:15 california_housing_test.csv
-rw-r--r-- 1 root root  1706430 Jun 23 01:15 california_housing_train.csv
-rw-r--r-- 1 root root 18289443 Jun 23 01:15 mnist_test.csv
-rw-r--r-- 1 root root 36523880 Jun 23 01:15 mnist_train_small.csv
-rwxr-xr-x 1 root root      930 Jan  1  2000 README.md

./small-file:
total 4
-rwxr-xr-x 1 root root 930 Jun 26 14:17 README.md


### Example - Getting unexpected function_call
In this scenario, the initial step went as anticipated, where the API provided the correct function and argument. However, following that, the API unexpectedly responded with a `python` function that includes code to identify the smallest file, which is unnecessary. It's important to note that we did not specify a `python` function anywhere.

Related issue - https://community.openai.com/t/function-calling-a-python-function-is-frequently-called-even-though-it-does-not-exist-in-the-functions-parameter/264481

In [ ]:
user_query = "Show me size of smallest file in sample data folder and copy that file to a new folder named test_copy"
answer = ask_openai(user_query)
print(answer)

INFO:OpenAI-Example:Calling API: [{'role': 'system', 'content': '\n                You are intelligent chatbot.\n                You are only allowed to use specified functions with relevant order by looking at the user query and context.\n                You don’t have a python function available, never attempt to call python\n                '}, {'role': 'user', 'content': 'Show me size of smallest file in sample data folder and copy that file to a new folder named test_copy'}]
INFO:OpenAI-Example:API Response: {'role': 'assistant', 'content': None, 'function_call': <OpenAIObject at 0x7f1cb4175ee0> JSON: {
  "name": "get_folder_contents",
  "arguments": "{\n  \"path\": \"/content/sample_data\"\n}"
}}
Sleeping: 100%|██████████| 30/30 [00:30<00:00,  1.00s/it]
INFO:OpenAI-Example:Calling API: [{'role': 'system', 'content': '\n                You are intelligent chatbot.\n                You are only allowed to use specified functions with relevant order by looking at the user query and 

JSONDecodeError: ignored